# Recreating Cogsci Prompts with NORMAL values vor off-variables
This notebook aims to re-create the prompts used in the cogsci submission where the off-variables are replaced with _normal_, e.g.,  high/low is replaced with high/normal. 


- Note, that in the in the original cogsci submission, the following things humans were presented with were not included and hence won't be included in this version:
    - no explanation following each causal mechanism
    - no Scaffolding, e.g. "variables"
    - no statement about the fact that each of the causes can independendently bring about the effect


In [1]:
import os
import sys
import pprint
import pandas as pd

# Ensure Python finds the `src` directory
sys.path.append(os.path.abspath("../../src"))

# Import everything defined in `__all__`
from causalalign.dataset_creation import (
    # rw_17_domain_components,
    graph_structures,
    inference_tasks_rw17,
    generate_prompt_dataframe,
    # expand_domain_to_dataframe,
    # expand_df_by_task_queries,
    create_domain_dict,
    # verbalize_domain_intro,
    # verbalize_causal_mechanism,
    # verbalize_inference_task,
    append_dfs,
)


In [2]:
rw_17_domain_components = {
    "economy": {
        "domain_name": "economy",
        "variables": {
            "X": {
                "name": "interest rates",
                "detailed": "Interest rates are the rates banks charge to loan money.",
                "p_value": {"1": "low", "0": "normal"},
                "m_value": {"1": "high", "0": "normal"},
                # "explanations": {
                #     "p_p": "Low interest rates stimulate economic growth, leading to greater prosperity overall, and allowing more money to be saved for retirement in particular.",
                #     "p_m": "The good economic times produced by the low interest rates leads to greater confidence and less worry about the future, so people are less concerned about retirement.",
                #     "m_p": "The high interest rates result in high yields on government bonds, which are especially attractive for retirement savings because they are such a safe investment.",
                #     "m_m": "A lot of people are making large monthly interest payments on credit card debt, and they have no money left to save for retirement.",
                # },
            },
            "Y": {
                "name": "trade deficits",
                "detailed": "A country's trade deficit is the difference between the value of the goods that a country imports and the value of the goods that a country exports.",
                "p_value": {"1": "small", "0": "normal"},
                "m_value": {"1": "large", "0": "normal"},
                # "explanations": {
                #     "p_p": "When the economy is good, people can cover their basic expenses and so have enough money left over to contribute to their retirement accounts.",
                #     "p_m": "When the economy is good, people are optimistic and so spend rather than save.",
                #     "m_p": "People become nervous when their economy is no longer competitive enough in the world economy to export products, and begin saving for retirement as a result.",
                #     "m_m": "The loss of local manufacturing jobs means that there are people out of work, and contributions to retirement accounts decreases.",
                # },
            },
            "Z": {
                "name": "retirement savings",
                "detailed": "Retirement savings is the money people save for their retirement.",
                "p_value": {"1": "high", "0": "normal"},
                "m_value": {"1": "low", "0": "normal"},
            },
        },
        "introduction": "Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these attributes are responsible for producing or causing one another.",
    },
    "sociology": {
        "domain_name": "sociology",
        "variables": {
            "X": {
                "name": "urbanization",
                "detailed": "Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.",
                "p_value": {"1": "high", "0": "normal"},
                "m_value": {"1": "low", "0": "normal"},
                # "explanations": {
                #     "p_p": "Big cities provide many opportunities for financial and social improvement.",
                #     "p_m": "In big cities many people are competing for the same high-status jobs and occupations.",
                #     "m_p": 'People in rural areas are rarely career oriented, and so take time off from working and switch frequently between different "temp" jobs.',
                #     "m_m": "The low density of people prevents the dynamic economic expansion needed for people to get ahead.",
                # },
            },
            "Y": {
                "name": "interest in religion",
                "detailed": "Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions.",
                "p_value": {"1": "low", "0": "normal"},
                "m_value": {"1": "high", "0": "normal"},
                # "explanations": {
                #     "p_p": "Without the restraint of religion-based morality, the impulse toward greed dominates and people tend to accumulate material wealth.",
                #     "p_m": "Many religions reinforce a strong work ethic; without this motivation, workers become complacent at their jobs.",
                #     "m_p": "Religion fosters communal care, and those of the same religion tend to support each other with jobs, financial favors, and so on.",
                #     "m_m": "The spiritualism induced by religion works to reduce the desire for material wealth.",
                # },
            },
            "Z": {
                "name": "socio-economic mobility",
                "detailed": "Socioeconomic mobility is the degree to which the members of a society are able to improve their social and economic status.",
                "p_value": {"1": "high", "0": "normal"},
                "m_value": {"1": "low", "0": "normal"},
            },
        },
        "introduction": "Sociologists seek to describe and predict the regular patterns of societal interactions. To do this, they study some important variables or attributes of societies. They also study how these attributes are responsible for producing or causing one another.",
    },
    "weather": {
        "domain_name": "weather",
        "variables": {
            "X": {
                "name": "ozone levels",
                "detailed": "Ozone is a gaseous allotrope of oxygen (O3) and is formed by exposure to UV radiation.",
                "p_value": {"1": "high", "0": "normal"},
                "m_value": {"1": "low", "0": "normal"},
                # Note: this is graph dependent. the explanations below are based on the collider graph
                # "explanations": {
                #     "p_m": "Ozone attracts extra oxygen atoms from water molecules, creating a concentration of water vapor in that region.",
                #     # "p_p": "Ozone accepts extra oxygen atoms, decreasing the amount of oxygen available to form water molecules. With fewer water molecules, there is lower humidity.",
                #     "p_p": "Ozone tends to rapidly break down into normal oxygen, leaving more oxygen available for the formation of water molecules (H2O).",
                #     "m_p": "The oxygen atoms that would normally be part of ozone molecules are free to combine with hydrogen atoms instead, creating water molecules.",
                #     "m_m": "The low amount of ozone allows a large number of ultra-violet (UV) rays to enter the atmosphere, and the UV rays break up water molecules, resulting in low humidity.",
                # },
            },
            "Y": {
                "name": "air pressure",
                "detailed": "Air pressure is force exerted due to concentrations of air molecules.",
                "p_value": {"1": "low", "0": "normal"},
                "m_value": {"1": "high", "0": "normal"},
                # Note: this is graph dependent. the explanations below are based on the collider graph
                # "explanations": {
                #     "p_p": "When pressure does not force water vapor to break into oxygen and hydrogen atoms, water vapor remains in abundance.",
                #     "p_m": "Low air pressure poorly facilitates condensation; as a result, there are less water molecules in the air.",
                #     "m_p": "The higher pressure means that the components of water molecules (hydrogen and oxygen) tend to not dissocciate from one another. Because there are more water molecules, humidity is higher.",
                #     "m_m": "When air pressure is high, water vapor condenses into liquid water (rain), and the atmosphere is left with little moisture.",
                # },
            },
            "Z": {
                "name": "humidity",
                "detailed": "Humidity is the degree to which the atmosphere contains water molecules.",
                "p_value": {"1": "high", "0": "normal"},
                "m_value": {"1": "low", "0": "normal"},
            },
        },
        "introduction": "Meteorologists seek to describe and predict the regular patterns that govern weather systems. To do this, they study some important variables or attributes of weather systems. They also study how these attributes are responsible for producing or causing one another.",
    },
}

In [3]:
econ_config_rw17 = rw_17_domain_components["economy"]["variables"]
socio_config_rw17 = rw_17_domain_components["sociology"]["variables"]
weather_config_rw17 = rw_17_domain_components["weather"]["variables"]
print("Domain components loaded successfully!")


print("Dataset creation module imported successfully!")

Domain components loaded successfully!
Dataset creation module imported successfully!


In [4]:
# load domain components
rw_17 = rw_17_domain_components


# list the domains in the dataset
print("Domains in the dataset:")
print(rw_17.keys())


# Pretty-print RW17 dataset structure

pprint.pprint(rw_17)

Domains in the dataset:
dict_keys(['economy', 'sociology', 'weather'])
{'economy': {'domain_name': 'economy',
             'introduction': 'Economists seek to describe and predict the '
                             'regular patterns of economic fluctuation. To do '
                             'this, they study some important variables or '
                             'attributes of economies. They also study how '
                             'these attributes are responsible for producing '
                             'or causing one another.',
             'variables': {'X': {'detailed': 'Interest rates are the rates '
                                             'banks charge to loan money.',
                                 'm_value': {'0': 'normal', '1': 'high'},
                                 'name': 'interest rates',
                                 'p_value': {'0': 'normal', '1': 'low'}},
                           'Y': {'detailed': "A country's trade deficit is the "
    

## Adjusted functions for cogsci purposes
For the purpose of re-creating the prompts in the original cogsci submission, we will re-define the functions normally imported, which contain some modifications. 
### 1. Create domain dict
- don't need explanation key
- don't need to re-define function, just don't provide explanation in config file which is input to this functoin

### 2. verbalize variables section
- exclude  from variables_text:  "Some xy have ... . others have ... ."
### 4. verbalize causal mechanism
- exclude explanation 
    - but that's already taken care of by not inlcuding explanations in the config in the first place

### 5. verbalize inference task

### 6. generate propmt datafranme

In [5]:
def verbalize_variables_section(domain_dict, row):
    """
    Create a comprehensive description of all variables based on counterbalance conditions.

    Parameters:
    -----------
    domain_dict : dict
        The domain dictionary containing variable information
    row : pd.Series
        Row from the expanded dataframe with counterbalance information

    Returns:
    --------
    str
        Formatted text describing all variables with their counterbalanced values
    """
    domain_name = domain_dict["domain_name"]

    # replace domain name such that it fits into the sentence
    if domain_name == "economy":
        domain_name = "economies"

    elif domain_name == "sociology":
        domain_name = "societies"

    elif domain_name == "weather":
        domain_name = "weather systems"

    variables_text = " "  # f"\n{domain_name}\n********** VARIABLES **********\n"

    for var_key, var_details in domain_dict["variables"].items():
        name = var_details[f"{var_key}_name"]
        detailed = var_details[f"{var_key}_detailed"]

        # Get appropriate value based on counterbalance
        cntbl = row[f"{var_key}_cntbl"]
        value_dict = var_details["p_value"] if cntbl == "p" else var_details["m_value"]

        # Create description with on/off values
        value_1 = value_dict["1"]
        value_0 = value_dict["0"]

        # Format the description
        variables_text += f"{detailed} "  # excluded for cogsci: Some {domain_name} have {value_1} {name}. Others have {value_0} {name}. "  # this should be different -> needed to reproduce exact RW17 prompts, but not needed for abstract prompts
        # else, for abstract prompts, it should be:
        # variables_text += f"{detailed} Sometimes {name} is {value_1} and sometimes{name} is {value_0}. "

    return variables_text

In [6]:
def verbalize_inference_task(row, nested_dict, prompt_type):
    """
    Generate the verbalized inference task based on a DataFrame row.

    Parameters:
    -----------
    row : pd.Series
        A row from the DataFrame containing the inference task information.
    nested_dict : dict
        The nested dictionary containing value mappings for variables created by `create_domain_dict`.
    prompt_type : str
        The type of prompt to be displayed to the LLM.

    Returns:
    --------
    str
        The verbalized inference task statement.
    """
    variables_dict = nested_dict["variables"]
    observations = row["observation"].split(", ")

    # Set domain-specific observation intro
    domain = row["domain"].lower()
    if domain == "weather":
        observation_text = (
            "Suppose that there is a weather system that is known to have "
        )
    elif domain == "economy":
        observation_text = "Suppose that the economy is currently known to have "
    elif domain == "sociology":
        observation_text = "Suppose that the society is known to have "
    else:
        observation_text = "You are currently observing: "

    observation_descriptions = []

    for obs in observations:
        var_name, value = obs.split("=")
        cntbl_type = row[f"{var_name}_cntbl"]

        if var_name not in variables_dict:
            raise KeyError(f"Variable {var_name} not found in nested_dict['variables']")

        value_mapping = variables_dict[var_name]

        if f"{cntbl_type}_value" in value_mapping:
            sense_label = value_mapping[f"{cntbl_type}_value"].get(value)
        else:
            available_mapping = value_mapping.get(
                "p_value", value_mapping.get("m_value")
            )
            sense_label = available_mapping.get(value)

        if sense_label is None:
            raise KeyError(
                f"Value {value} for {var_name} is missing in nested_dict['variables'][{var_name}]"
            )

        observation_descriptions.append(f"{sense_label} {row[var_name]}")

    observation_text += " and ".join(observation_descriptions) + "."

    # Extract the query variable and construct the query
    query_var = row["query_node"].split("=")[0]
    query_sense = row[f"{query_var}_sense"]
    query_text = f" Your task is to estimate how likely it is that {query_sense} {row[query_var]} are present on a scale from 0 to 100, given the observations and causal relationships described. 0 means completely unlikely and 100 means completely certain. {prompt_type}"

    return observation_text + query_text


Now, let's explore how these components are combined by re-creating the prompts used in RW17 starting with re-creating the dictionaries that are stored in ``constants.py``!


# Step 1: Create Domain Dictionray:


In [7]:
# # Example usage with enforcement of "normal" for zero values
# try:
#     economy_test_dict = create_domain_dict(
#         domain="economy",
#         introduction="Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these variables are responsible for producing or causing one another.",
#         X_name="interest rates",
#         X_detailed="Interest rates are the rates banks charge to loan money.",
#         X_range_intro =f"Some economies have <tag> interest rates. Others have <tag> interest rates."
#         X_values={"1": "low", "0": "high"},
#         Y_name="trade deficits",
#         Y_detailed="A country's trade deficit...",
#         Y_values={"1": "small", "0": "large"},
#         Z_name="retirement savings",
#         Z_detailed="Retirement savings is the money people save for their retirement.",
#         Z_values={"1": "high", "0": "low"},
#         counterbalance_enabled=True,
#         enforce_zero_label=True,  # Ensures '0' is verbalized as 'normal'
#         zero_label="normal",
#     )

#     # Convert dictionary to a readable JSON format for display
#     import json

#     formatted_output = json.dumps(economy_test_dict, indent=4)
#     print(formatted_output)

# except Exception as e:
#     print(f"Error: {e}")

## Create Domain dicts

In [8]:
econ_dict_rw17 = create_domain_dict(
    domain_name="economy",
    introduction="Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these attributes are responsible for producing or causing one another.",
    variables_config=econ_config_rw17,
    graph_type="collider",
)
# Generate prompts based on dict created in this notebook
econ_prompts_df = generate_prompt_dataframe(
    econ_dict_rw17, inference_tasks_rw17, "collider", graph_structures
)
print(f"shape: {econ_prompts_df.shape}")


weather_dict_rw17 = create_domain_dict(
    domain_name="weather",
    introduction="Meteorologists seek to describe and predict the regular patterns that govern weather systems. To do this, they study some important variables or attributes of weather systems. They also study how these attributes are responsible for producing or causing one another.",
    variables_config=weather_config_rw17,
    graph_type="collider",
)
# Generate prompts based on dict created in this notebook
weather_prompts_df = generate_prompt_dataframe(
    weather_dict_rw17, inference_tasks_rw17, "collider", graph_structures
)
weather_prompts_df.shape

sociology_dict_rw17 = create_domain_dict(
    domain_name="sociology",
    introduction="Sociologists seek to describe and predict the regular patterns of societal interactions. To do this, they study some important variables or attributes of societies. They also study how these attributes are responsible for producing or causing one another.",
    variables_config=socio_config_rw17,
    graph_type="collider",
)
# Generate prompts based on dict created in this notebook
sociology_prompts_df = generate_prompt_dataframe(
    sociology_dict_rw17, inference_tasks_rw17, "collider", graph_structures
)
sociology_prompts_df.shape

shape: (160, 24)


(160, 24)

Now append the individual dataframes

In [9]:
rw_17_over_complete_df = append_dfs(
    econ_prompts_df, weather_prompts_df, sociology_prompts_df
)
# add a unique id to each row
rw_17_over_complete_df["id"] = range(1, len(rw_17_over_complete_df) + 1)


# subset for the counterbalance conditions used in the RW17 study
select_contbl_cond_xs = ["ppp", "pmm", "mmp", "mpm"]
print(
    f"unique counterbalance conditions used: {rw_17_over_complete_df['cntbl_cond'].unique()}"
)

rw_17_complete_df = rw_17_over_complete_df[
    rw_17_over_complete_df["cntbl_cond"].isin(select_contbl_cond_xs)
]


unique counterbalance conditions used: ['ppp' 'ppm' 'pmp' 'pmm' 'mpp' 'mpm' 'mmp' 'mmm']


In [10]:
# this has already been saved in the collider numeric-certainty notebook
# drop text verbalization columns from merged df
# merged_df = merged_df.drop(
#     columns=["C1_detailed", "C2_detailed", "E_detailed", "prompt"]
# )
# # save merged data
# merged_df.to_csv(
#     f"../../../results/17_rw/humans/humans_responses_w_prompt_id_{graph_type}_{prompt_type}.csv",
#     index=False,
#     sep=";",
# )
# print(merged_df.columns)


### Saving the Prompts for LLMs

Numeric-only version

In [11]:
version = "6"
# save for prompting LLMs
LLM_prompting_df = rw_17_complete_df[
    ["id", "prompt", "prompt_category", "graph", "domain", "cntbl_cond", "task"]
]
prompt_category = LLM_prompting_df["prompt_category"].unique()[0]
graph_type = LLM_prompting_df["graph"].unique()[0]
print(prompt_category, graph_type)
# LLM_prompting_df.to_csv(
#     f"../datasets/17_rw/prompts_for_LLM_api/{version}_v_{prompt_category}_LLM_prompting_{graph_type}.csv",
#     index=False,
# )

print(
    f"Prompts for LLMs for version {version}, graph = {graph_type}, and prompt type = {prompt_category} saved successfully!"
)
print("prompt data file has the following columns:")
print(LLM_prompting_df.columns)

single_numeric_response collider
Prompts for LLMs for version 6, graph = collider, and prompt type = single_numeric_response saved successfully!
prompt data file has the following columns:
Index(['id', 'prompt', 'prompt_category', 'graph', 'domain', 'cntbl_cond',
       'task'],
      dtype='object')


### New Human Collider Data (non-aggregated)

In [12]:
human_data_new_ce = pd.read_csv(
    "../data/17_rw/human_data/rw17_collider_ce.csv", sep=";"
)
# rename colum task as label
human_data_new_ce.rename(columns={"letter.type": "label"}, inplace=True)
# print the columns
# turn label into lower case
human_data_new_ce["label"] = human_data_new_ce["label"].str.lower()
# rename attr.polarity to ppp
human_data_new_ce.rename(columns={"attr.polarity": "ppp"}, inplace=True)
# drop task
human_data_new_ce.drop(columns=["task"], inplace=True)
print(human_data_new_ce.columns)

print(human_data_new_ce["label"].unique())
human_data_new_ce.head()


Index(['s', 'domain', 'diagram', 'ppp', 'label', 'study.type', 'trial', 'rt',
       'y', 'type', 'aggr.type', 'betw.factors'],
      dtype='object')
['d' 'j' 'b' 'f' 'c' 'k' 'h' 'e' 'a' 'g' 'i']


,s,domain,diagram,ppp,label,study.type,trial,rt,y,type,aggr.type,betw.factors
0,0,economy,a,ppp,d,Y|X=1,0,"38,788",65,CB|CA=1,Ci|Cj=1,"domain, attr.polarity, diagram, letter.type"
1,0,economy,a,ppp,j,"Z|X=0,Y=1",1,"16,532",80,"E|CA=0,CB=1","E|Ci=0,Cj=1","domain, attr.polarity, diagram, letter.type"
2,0,economy,a,ppp,b,X|Z=1,2,"10,979",90,CA|E=1,Ci|E=1,"domain, attr.polarity, diagram, letter.type"
3,0,economy,a,ppp,f,"X|Y=1,Z=0",3,"17,956",55,"CA|E=0,CB=1","Ci|E=0,Cj=1","domain, attr.polarity, diagram, letter.type"
4,0,economy,a,ppp,c,"Y|X=0,Z=1",4,"9,54",60,"CB|E=1,CA=0","Ci|E=1,Cj=0","domain, attr.polarity, diagram, letter.type"


### we need to get the human data into the followin format (mostly column renaming):

(['human_subj_id', 'domain', 'cntbl_cond', 'response',
       'num_responses_agg', 'task', 'subject', 'graph'],
      dtype='object')

In [13]:
# rename the columns
human_data_new_ce.rename(
    columns={
        "y": "response",
        "label": "task",
        "s": "human_subj_id",
        "ppp": "cntbl_cond",
    },
    inplace=True,
)

# and drop all other columns

human_data_new_ce = human_data_new_ce[
    [
        "response",
        "task",
        "human_subj_id",
        "cntbl_cond",
        # "type",
        "domain",
    ]
]


# renaming and new columns
human_data_new_ce["domain"] = human_data_new_ce["domain"].replace(
    {"society": "sociology"}
)

human_data_new_ce["subject"] = "human"
# add a column graph that contains collider everywhere
human_data_new_ce["graph"] = "collider"


# turn response into floats

human_data_new_ce["response"] = (
    human_data_new_ce["response"].replace(",", ".", regex=True).astype(float)
)

In [14]:
print(f"uniue domain values: {human_data_new_ce['domain'].unique()}")
print(f"uniue cntbl_cond values: {human_data_new_ce['cntbl_cond'].unique()}")
print(f"uniue task values: {human_data_new_ce['task'].unique()}")
# print(f"uniue type values: {human_data_new_ce['type'].unique()}")
print(f" unique response values: {human_data_new_ce['response'].unique()}")
# save human data
human_data_new_ce.head()

uniue domain values: ['economy' 'sociology' 'weather']
uniue cntbl_cond values: ['ppp' 'pmm' 'mmp' 'mpm']
uniue task values: ['d' 'j' 'b' 'f' 'c' 'k' 'h' 'e' 'a' 'g' 'i']
 unique response values: [ 65.  80.  90.  55.  60.  75.  85.  45.  50.  95.  40.  30.  20. 100.
  70.  15.   0.  10.  35.  25.   5.]


,response,task,human_subj_id,cntbl_cond,domain,subject,graph
0,65.0,d,0,ppp,economy,human,collider
1,80.0,j,0,ppp,economy,human,collider
2,90.0,b,0,ppp,economy,human,collider
3,55.0,f,0,ppp,economy,human,collider
4,60.0,c,0,ppp,economy,human,collider


In [15]:
print(f"NEW: human data shape: {human_data_new_ce.shape}")
# print(f"Old human data shape: {human_data.shape}")

NEW: human data shape: (960, 7)


### Merge new human data (non-aggregated) with the prompts

In [16]:
# now merge the all_domains_df with  human data on the columns: domain, task, cntbl_cond
# merge the dataframes
merged_df_new = pd.merge(
    rw_17_complete_df, human_data_new_ce, on=["domain", "task", "cntbl_cond", "graph"]
)
# merged_df_new["response"] = merged_df_new["response"].str.replace(",", ".").astype(float)
# print the columns
print(merged_df_new.columns)

Index(['domain', 'X', 'X_values', 'X_cntbl', 'X_sense', 'X_detailed', 'Y',
       'Y_values', 'Y_cntbl', 'Y_sense', 'Y_detailed', 'Z', 'Z_values',
       'Z_cntbl', 'Z_sense', 'Z_detailed', 'cntbl_cond', 'task', 'query_node',
       'observation', 'query', 'graph', 'prompt', 'prompt_category', 'id',
       'response', 'human_subj_id', 'subject'],
      dtype='object')


In [17]:
merged_df_new
# drop duplicate rows
merged_df_new = merged_df_new.drop_duplicates()

In [18]:
merged_df_new[merged_df_new["id"] == 66]

,domain,X,X_values,X_cntbl,X_sense,X_detailed,Y,Y_values,Y_cntbl,Y_sense,...,query_node,observation,query,graph,prompt,prompt_category,id,response,human_subj_id,subject
202,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,100.0,13,human
204,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,90.0,40,human
205,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,60.0,40,human
206,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,20.0,112,human
207,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,70.0,112,human
208,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,40.0,119,human
209,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,45.0,119,human
210,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,m,large,...,Y=1,"Z=1, X=0","p(Y=1|Z=1, X=0)",collider,Economists seek to describe and predict the re...,single_numeric_response,66,85.0,133,human


In [19]:
# print are there any duplicates in the merged_df_new
duplicates = merged_df_new[merged_df_new.duplicated()]
# print(f"duplicates: {duplicates}")
# print the number of duplicates
print(f"number of duplicates: {len(duplicates)}")
# print the number of rows in the merged_df_new
# print(f"number of rows in the merged_df: {len(merged_df)}")
# print the number of rows in the merged_df
print(f"number of rows in the merged_df_new: {len(merged_df_new)}")


number of duplicates: 0
number of rows in the merged_df_new: 1540


In [20]:
# rename reasoning_type

In [21]:
duplicates

,domain,X,X_values,X_cntbl,X_sense,X_detailed,Y,Y_values,Y_cntbl,Y_sense,...,query_node,observation,query,graph,prompt,prompt_category,id,response,human_subj_id,subject


In [ ]:
# for each column, print the number of unique values
for col in merged_df_new.columns:
    print(
        f"{col}:  {merged_df_new[col].nunique()} unique values: {merged_df_new[col].unique()}"
    )


merged_df_new.head()


domain:  3 unique values: ['economy' 'weather' 'sociology']
X:  3 unique values: ['interest rates' 'ozone levels' 'urbanization']
X_values:  1 unique values: ['1']
X_cntbl:  2 unique values: ['p' 'm']
X_sense:  2 unique values: ['low' 'high']
X_detailed:  3 unique values: ['Interest rates are the rates banks charge to loan money.'
 'Ozone is a gaseous allotrope of oxygen (O3) and is formed by exposure to UV radiation.'
 'Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.']
Y:  3 unique values: ['trade deficits' 'air pressure' 'interest in religion']
Y_values:  1 unique values: ['1']
Y_cntbl:  2 unique values: ['p' 'm']
Y_sense:  4 unique values: ['small' 'large' 'low' 'high']
Y_detailed:  3 unique values: ["A country's trade deficit is the difference between the value of the goods that a country imports and the value of the goods that a country exports."
 'Air pressure is force exerted due to concentrations 

,domain,X,X_values,X_cntbl,X_sense,X_detailed,Y,Y_values,Y_cntbl,Y_sense,...,query_node,observation,query,graph,prompt,prompt_category,id,response,human_subj_id,subject
0,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,p,small,...,X=1,"Z=1, Y=1","p(X=1|Z=1, Y=1)",collider,Economists seek to describe and predict the re...,single_numeric_response,1,80.0,0,human
1,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,p,small,...,X=1,"Z=1, Y=1","p(X=1|Z=1, Y=1)",collider,Economists seek to describe and predict the re...,single_numeric_response,1,85.0,0,human
2,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,p,small,...,X=1,"Z=1, Y=1","p(X=1|Z=1, Y=1)",collider,Economists seek to describe and predict the re...,single_numeric_response,1,75.0,25,human
3,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,p,small,...,X=1,"Z=1, Y=1","p(X=1|Z=1, Y=1)",collider,Economists seek to describe and predict the re...,single_numeric_response,1,70.0,25,human
4,economy,interest rates,1,p,low,Interest rates are the rates banks charge to l...,trade deficits,1,p,small,...,X=1,"Z=1, Y=1","p(X=1|Z=1, Y=1)",collider,Economists seek to describe and predict the re...,single_numeric_response,1,65.0,58,human


In [ ]:
# where subject = human, replace the value in the column prompt_type with single_numeric_response
merged_df_new.loc[merged_df_new["subject"] == "human", "prompt_type"] = (
    "single_numeric_response"
)


# print unique values in the column prompt_type
print(
    f"unique values in the column prompt_type: {merged_df_new['prompt_type'].unique()}"
)

unique values in the column prompt_type: ['single_numeric_response']


### Save complete raw datafrane containing all the columns


In [ ]:
raw_detailed_info = merged_df_new.copy()


# save merged data
raw_detailed_info.to_csv(
    f"../../../results/17_rw/humans/detailed_humans_responses_w_prompt_id_{graph_type}_{prompt_category}.csv",
    index=False,
    sep=";",
)

In [25]:
# save merged data
# merged_df.to_csv("../datasets/17_rw/merged_data/humans_prompts_coll.csv", index=False)
# print(merged_df.columns)

# merged_df_new.to_csv(
#     "../../../results/17_rw/humans/humans_responses_w_prompt_id_coll.csv",
#     index=False,
#     sep=";",
# )
print(merged_df_new.columns)


Index(['domain', 'X', 'X_values', 'X_cntbl', 'X_sense', 'X_detailed', 'Y',
       'Y_values', 'Y_cntbl', 'Y_sense', 'Y_detailed', 'Z', 'Z_values',
       'Z_cntbl', 'Z_sense', 'Z_detailed', 'cntbl_cond', 'task', 'query_node',
       'observation', 'query', 'graph', 'prompt', 'prompt_category', 'id',
       'response', 'human_subj_id', 'subject', 'prompt_type'],
      dtype='object')


## next, save LLM version (prompts)


In [26]:
version = "6"
# save for prompting LLMs
LLM_prompting_df = rw_17_complete_df[
    ["id", "prompt", "prompt_category", "graph", "domain", "cntbl_cond", "task"]
]
prompt_category = LLM_prompting_df["prompt_category"].unique()[0]
graph_type = LLM_prompting_df["graph"].unique()[0]
print(prompt_category, graph_type)
LLM_prompting_df.to_csv(
    f"../datasets/17_rw/prompts_for_LLM_api/{version}_v_{prompt_category}_LLM_prompting_{graph_type}.csv",
    index=False,
)

print(
    f"Prompts for LLMs for version {version}, graph = {graph_type}, and prompt type = {prompt_category} saved successfully!"
)
print("prompt data file has the following columns:")
print(LLM_prompting_df.columns)

single_numeric_response collider
Prompts for LLMs for version 6, graph = collider, and prompt type = single_numeric_response saved successfully!
prompt data file has the following columns:
Index(['id', 'prompt', 'prompt_category', 'graph', 'domain', 'cntbl_cond',
       'task'],
      dtype='object')


In [27]:
LLM_prompting_df.head()

,id,prompt,prompt_category,graph,domain,cntbl_cond,task
0,1,Economists seek to describe and predict the re...,single_numeric_response,collider,economy,ppp,a
1,2,Economists seek to describe and predict the re...,single_numeric_response,collider,economy,ppp,a
2,3,Economists seek to describe and predict the re...,single_numeric_response,collider,economy,ppp,b
3,4,Economists seek to describe and predict the re...,single_numeric_response,collider,economy,ppp,b
4,5,Economists seek to describe and predict the re...,single_numeric_response,collider,economy,ppp,c


76,"Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these attributes are responsible for producing or causing one another. Here are some variables: Interest rates are the rates banks charge to loan money. Some economies have low interest rates. Others have normal interest rates. A country's trade deficit is the difference between the value of the goods that a country imports and the value of the goods that a country exports. Some economies have large trade deficits. Others have normal trade deficits. Retirement savings is the money people save for their retirement. Some economies have low retirement savings. Others have normal retirement savings. Here are the causal relationships:  low interest rates cause low retirement savings.  large trade deficits cause low retirement savings. Suppose that the economy is currently known to have normal retirement savings and normal interest rates. Your task is to estimate how likely it is that large trade deficits are present on a scale from 0 to 100, given the observations and causal relationships described. 0 means completely unlikely and 100 means completely certain.  Return your response as raw text in one single line using this exact XML format: <response><likelihood>YOUR_NUMERIC_RESPONSE_HERE</likelihood></response> Replace YOUR_NUMERIC_RESPONSE_HERE with your likelihood estimate between 0 (very unlikely) and 100 (very likely). DO NOT include any other information, explanation, or formatting in your response. DO NOT use Markdown, code blocks, quotation marks, or special characters.",single_numeric_response,collider,economy,pmm,h




394,"Sociologists seek to describe and predict the regular patterns of societal interactions. To do this, they study some important variables or attributes of societies. They also study how these attributes are responsible for producing or causing one another. Here are some variables: Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments. Some societies have high urbanization. Others have normal urbanization. Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions. Some societies have high interest in religion. Others have normal interest in religion. Socioeconomic mobility is the degree to which the members of a society are able to improve their social and economic status. Some societies have low socio-economic mobility. Others have normal socio-economic mobility. Here are the causal relationships:  high urbanization cause low socio-economic mobility.  high interest in religion cause low socio-economic mobility. Suppose that the society you live in currently exhibits the following: normal socio-economic mobility. Your task is to estimate how likely it is that high interest in religion are present on a scale from 0 to 100, given the observations and causal relationships described. 0 means completely unlikely and 100 means completely certain.  Return your response as raw text in one single line using this exact XML format: <response><likelihood>YOUR_NUMERIC_RESPONSE_HERE</likelihood></response> Replace YOUR_NUMERIC_RESPONSE_HERE with your likelihood estimate between 0 (very unlikely) and 100 (very likely). DO NOT include any other information, explanation, or formatting in your response. DO NOT use Markdown, code blocks, quotation marks, or special characters.",single_numeric_response,collider,sociology,pmm,g

